In [74]:
import pandas as pd
from hmmlearn import hmm
import numpy as np
from collections import Counter


In [75]:
df = pd.read_csv('training_dataset.csv')

grouped_data = df.groupby(['id', df.groupby('id')['laneId'].transform('first')])

data_dict = {}

for group_name, group_data in grouped_data:
    data_dict[group_name] = group_data

selected_groups2 = grouped_data.filter(lambda x: x['laneId'].iloc[0] == 2)
selected_groups3 = grouped_data.filter(lambda x: x['laneId'].iloc[0] == 3)
selected_groups5 = grouped_data.filter(lambda x: x['laneId'].iloc[0] == 5)
selected_groups6 = grouped_data.filter(lambda x: x['laneId'].iloc[0] == 6)

train_data2 = selected_groups2[['x', 'y', 'xVelocity', 'yVelocity', 'state']].values
train_data3 = selected_groups3[['x', 'y', 'xVelocity', 'yVelocity', 'state']].values
train_data5 = selected_groups5[['x', 'y', 'xVelocity', 'yVelocity', 'state']].values
train_data6 = selected_groups6[['x', 'y', 'xVelocity', 'yVelocity', 'state']].values

print(train_data2)
num_rows = len(train_data2)

print(num_rows)


[[ 1.6275e+02  9.3900e+00 -3.2040e+01  0.0000e+00  2.0000e+00]
 [ 1.6158e+02  9.3900e+00 -3.2060e+01  0.0000e+00  2.0000e+00]
 [ 1.6036e+02  9.3900e+00 -3.2070e+01  0.0000e+00  2.0000e+00]
 ...
 [ 3.1873e+02  9.3600e+00 -2.9460e+01  2.4000e-01  2.0000e+00]
 [ 3.1754e+02  9.3700e+00 -2.9450e+01  2.5000e-01  2.0000e+00]
 [ 3.1636e+02  9.3800e+00 -2.9440e+01  2.5000e-01  2.0000e+00]]
93546


In [76]:
df = pd.read_csv('train_dataset2c.csv')

train_data2c = df[['x', 'y', 'xVelocity', 'yVelocity', 'state']].values

df = pd.read_csv('train_dataset6c.csv')

train_data6c = df[['x', 'y', 'xVelocity', 'yVelocity', 'state']].values


In [113]:
model2c = hmm.GaussianHMM(n_components=4, covariance_type="full", n_iter=10000)

model2c.fit(train_data2c)

print("Training Finish!")

Training Finish!


In [114]:
print(model2c.means_)

[[ 2.62695385e+02  9.73051282e+00 -3.62453846e+01  3.46923077e-01
   2.00000000e+00]
 [ 4.88581579e+01  1.32206579e+01 -3.64794737e+01  2.50657895e-01
   4.00000000e+00]
 [ 1.55626429e+02  1.16517143e+01 -3.65504286e+01  7.62857143e-01
   3.00000000e+00]
 [ 2.62701842e+02  9.72447368e+00 -3.62463158e+01  3.46842105e-01
   2.00000000e+00]]


In [137]:
right = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
error = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
for i in range(10000):
    # 'x', 'y', 'xVelocity', 'yVelocity', 'state'
    current_data = [train_data2[i+2]]
    history_data1 = [train_data2[i]]
    history_data2 = [train_data2[i+1]]
    #print(history_data1,history_data2,current_data,)
    for j in range(10):        
        predicted_state = model2c.predict(current_data)
        #print('predict state is:',predicted_state)
        posterior_probs = model2c.predict_proba(current_data)
        most_probable_state = np.argmax(posterior_probs)
        predicted_observation = model2c.means_[most_probable_state]
        if (predicted_state == [3]):
            predict_result = current_data[0][4] 
        else:
            predict_result = predicted_observation[4]
        if(predict_result==int(train_data2[i+j+3][4])):
            right[j] = right[j] + 1
        else:
            error[j] = error[j] + 1
        
        next_vx = current_data[0][2] + ((current_data[0][2] - history_data2[0][2]) - (history_data2[0][2] - history_data1[0][2]))
        next_vy = current_data[0][3] + ((current_data[0][3] - history_data2[0][3]) - (history_data2[0][3] - history_data1[0][3]))
        next_x = 0.04 * 0.5 * (next_vx + current_data[0][2]) + current_data[0][0]
        next_y = 0.04 * 0.5 * (next_vy + current_data[0][3]) + current_data[0][1]

        current_data = [[next_x, next_y, next_vx, next_vy, predict_result]]
        history_data1 = history_data2
        history_data2 = current_data
        #print('Predict result is: ', current_data)
        #print('Real result is:', train_data2[i+j+3])

        
print('Right Num:', right, 'Error Num:', error)

Right Num: [9841, 9796, 9751, 9706, 9661, 9619, 9576, 9536, 9494, 9451] Error Num: [159, 204, 249, 294, 339, 381, 424, 464, 506, 549]


In [138]:
right = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
error = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
for i in range(10000):
    # 'x', 'y', 'xVelocity', 'yVelocity', 'state'
    current_data = [train_data3[i+2]]
    history_data1 = [train_data3[i]]
    history_data2 = [train_data3[i+1]]
    #print(history_data1,history_data2,current_data,)
    for j in range(10):        
        predicted_state = model2c.predict(current_data)
        #print('predict state is:',predicted_state)
        posterior_probs = model2c.predict_proba(current_data)
        most_probable_state = np.argmax(posterior_probs)
        predicted_observation = model2c.means_[most_probable_state]
        if (predicted_state == [3]):
            predict_result = current_data[0][4] 
        else:
            predict_result = predicted_observation[4]
        if(predict_result==int(train_data3[i+j+3][4])):
            right[j] = right[j] + 1
        else:
            error[j] = error[j] + 1
        
        next_vx = current_data[0][2] + ((current_data[0][2] - history_data2[0][2]) - (history_data2[0][2] - history_data1[0][2]))
        next_vy = current_data[0][3] + ((current_data[0][3] - history_data2[0][3]) - (history_data2[0][3] - history_data1[0][3]))
        next_x = 0.04 * 0.5 * (next_vx + current_data[0][2]) + current_data[0][0]
        next_y = 0.04 * 0.5 * (next_vy + current_data[0][3]) + current_data[0][1]

        current_data = [[next_x, next_y, next_vx, next_vy, predict_result]]
        history_data1 = history_data2
        history_data2 = current_data
        #print('Predict result is: ', current_data)
        #print('Real result is:', train_data3[i+j+3])

        
print('Right Num:', right, 'Error Num:', error)

Right Num: [8906, 8880, 8856, 8836, 8816, 8795, 8776, 8755, 8736, 8714] Error Num: [1094, 1120, 1144, 1164, 1184, 1205, 1224, 1245, 1264, 1286]


In [130]:
df = pd.read_csv('train_dataset6c.csv')

train_data6c = df[['x', 'y', 'xVelocity', 'yVelocity', 'state']].values
print(train_data6c)
model6c = hmm.GaussianHMM(n_components=4, covariance_type="full", n_iter=10000)

model6c.fit(train_data6c)

print("Training Finish!")
print(model6c.means_)


[[ 1.8150e+02  2.5490e+01  2.5070e+01  9.0000e-02  7.0000e+00]
 [ 1.8239e+02  2.5490e+01  2.5060e+01  9.0000e-02  7.0000e+00]
 [ 1.8333e+02  2.5490e+01  2.5050e+01  8.0000e-02  7.0000e+00]
 ...
 [ 4.1111e+02  2.1880e+01  2.6840e+01 -1.2000e-01  5.0000e+00]
 [ 4.1219e+02  2.1870e+01  2.6870e+01 -1.0000e-01  5.0000e+00]
 [ 4.1326e+02  2.1870e+01  2.6900e+01 -9.0000e-02  5.0000e+00]]
Training Finish!
[[ 2.85718305e+02  2.53101094e+01  2.39499794e+01 -1.55104491e-01
   7.00000000e+00]
 [ 2.18809839e+02  2.54403511e+01  2.46173407e+01 -4.22173880e-02
   7.00000000e+00]
 [ 3.87899592e+02  2.22387755e+01  2.61804082e+01 -5.54897959e-01
   5.00000000e+00]
 [ 3.38506458e+02  2.40229167e+01  2.47018750e+01 -1.05020833e+00
   6.00000000e+00]]


In [139]:
right = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
error = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
for i in range(10000):
    # 'x', 'y', 'xVelocity', 'yVelocity', 'state'
    current_data = [train_data6[i+2]]
    history_data1 = [train_data6[i]]
    history_data2 = [train_data6[i+1]]
    for j in range(10):        
        predicted_state = model6c.predict(current_data)
        #print('predict state is:',predicted_state)
        posterior_probs = model6c.predict_proba(current_data)
        most_probable_state = np.argmax(posterior_probs)
        predicted_observation = model6c.means_[most_probable_state]
        if (predicted_state == [1]):
            predict_result = current_data[0][4] 
        else:
            predict_result = predicted_observation[4]
        if(predict_result==int(train_data6[i+j+3][4])):
            right[j] = right[j] + 1
        else:
            error[j] = error[j] + 1
        
        next_vx = current_data[0][2] + ((current_data[0][2] - history_data2[0][2]) - (history_data2[0][2] - history_data1[0][2]))
        next_vy = current_data[0][3] + ((current_data[0][3] - history_data2[0][3]) - (history_data2[0][3] - history_data1[0][3]))
        next_x = 0.04 * 0.5 * (next_vx + current_data[0][2]) + current_data[0][0]
        next_y = 0.04 * 0.5 * (next_vy + current_data[0][3]) + current_data[0][1]

        current_data = [[next_x, next_y, next_vx, next_vy, predict_result]]
        history_data1 = history_data2
        history_data2 = current_data
        #print('Predict result is: ', current_data)
        #print('Real result is:', train_data6[i+j+3])

        
print('Right Num:', right, 'Error Num:', error)

Right Num: [9936, 9882, 9834, 9786, 9734, 9680, 9629, 9584, 9535, 9486] Error Num: [64, 118, 166, 214, 266, 320, 371, 416, 465, 514]


In [140]:
right = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
error = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
for i in range(10000):
    # 'x', 'y', 'xVelocity', 'yVelocity', 'state'
    current_data = [train_data5[i+2]]
    history_data1 = [train_data5[i]]
    history_data2 = [train_data5[i+1]]
    #print(history_data1,history_data2,current_data,)
    for j in range(10):        
        predicted_state = model6c.predict(current_data)
        #print('predict state is:',predicted_state)
        posterior_probs = model6c.predict_proba(current_data)
        most_probable_state = np.argmax(posterior_probs)
        predicted_observation = model6c.means_[most_probable_state]
        if (predicted_state == [1]):
            predict_result = current_data[0][4] 
        else:
            predict_result = predicted_observation[4]
        if(predict_result==int(train_data5[i+j+3][4])):
            right[j] = right[j] + 1
        else:
            error[j] = error[j] + 1
        
        next_vx = current_data[0][2] + ((current_data[0][2] - history_data2[0][2]) - (history_data2[0][2] - history_data1[0][2]))
        next_vy = current_data[0][3] + ((current_data[0][3] - history_data2[0][3]) - (history_data2[0][3] - history_data1[0][3]))
        next_x = 0.04 * 0.5 * (next_vx + current_data[0][2]) + current_data[0][0]
        next_y = 0.04 * 0.5 * (next_vy + current_data[0][3]) + current_data[0][1]

        current_data = [[next_x, next_y, next_vx, next_vy, predict_result]]
        history_data1 = history_data2
        history_data2 = current_data
        #print('Predict result is: ', current_data)
        #print('Real result is:', train_data5[i+j+3])

        
print('Right Num:', right, 'Error Num:', error)

Right Num: [9964, 9928, 9892, 9856, 9820, 9784, 9748, 9713, 9678, 9643] Error Num: [36, 72, 108, 144, 180, 216, 252, 287, 322, 357]
